# Understanding Caching

In [3]:
import findspark
findspark.init()

In [4]:
from pyspark import SparkConf
from pyspark.sql import SparkSession
conf = SparkConf() \
        .setAppName("Dyanmic Partitions") \
        .set("spark.driver.memory", "4g") \
        .set("spark.driver.cores", "5") \
        .set("spark.master", "local[5]") \

spark = SparkSession.builder.config(conf=conf).getOrCreate()

## Create Dataframes

In [6]:
ratingDF = spark.read.option("header","true").csv("D://TechnoAvengers//DI//Spark_Optimizations//ml-25m//ratings.csv")
tagsDF = spark.read.option("header","true").csv("D://TechnoAvengers//DI//Spark_Optimizations//ml-25m//tags.csv")

## Joining and group Dataset without Caching

In [ ]:
from pyspark.sql.functions import *
joinDF = ratingDF.join(broadcast(tagsDF),"movieId")
joinDF.cache()
joinDF.show()

In [ ]:
joinDF = ratingDF.join(broadcast(tagsDF),"movieId")
joinDF.groupBy("rating").count().write.mode("overwrite").saveAsTable("rating_cnt1")
joinDF.groupBy("rating").avg().write.mode("overwrite").saveAsTable("rating_avg1")

## Add Caching

In [7]:
from pyspark.sql.functions import *
joinDF = ratingDF.join(broadcast(tagsDF),"movieId")
joinDF.cache().show()

Py4JJavaError: An error occurred while calling o60.showString.
: org.apache.spark.SparkException: Job 3 cancelled 
	at org.apache.spark.scheduler.DAGScheduler.failJobAndIndependentStages(DAGScheduler.scala:2059)
	at org.apache.spark.scheduler.DAGScheduler.handleJobCancellation(DAGScheduler.scala:1955)
	at org.apache.spark.scheduler.DAGSchedulerEventProcessLoop.doOnReceive(DAGScheduler.scala:2205)
	at org.apache.spark.scheduler.DAGSchedulerEventProcessLoop.onReceive(DAGScheduler.scala:2188)
	at org.apache.spark.scheduler.DAGSchedulerEventProcessLoop.onReceive(DAGScheduler.scala:2177)
	at org.apache.spark.util.EventLoop$$anon$1.run(EventLoop.scala:49)
	at org.apache.spark.scheduler.DAGScheduler.runJob(DAGScheduler.scala:775)
	at org.apache.spark.SparkContext.runJob(SparkContext.scala:2114)
	at org.apache.spark.SparkContext.runJob(SparkContext.scala:2135)
	at org.apache.spark.SparkContext.runJob(SparkContext.scala:2154)
	at org.apache.spark.sql.execution.SparkPlan.executeTake(S

In [ ]:
joinDF.count().write.mode("overwrite").saveAsTable("rating_cnt")
joinDF.avg().write.mode("overwrite").saveAsTable("rating_avg")

In [4]:
from pyspark.sql.functions import *
import pyspark
joinDF = ratingDF.join(broadcast(tagsDF),"movieId")
joinDF.persist(pyspark.StorageLevel.OFF_HEAP)

joinDF.count().write.mode("overwrite").saveAsTable("rating_cnt")
joinDF.avg().write.mode("overwrite").saveAsTable("rating_avg")

Py4JJavaError: An error occurred while calling o58.count.
: org.apache.spark.SparkException: Job 3 cancelled 
	at org.apache.spark.scheduler.DAGScheduler.failJobAndIndependentStages(DAGScheduler.scala:2059)
	at org.apache.spark.scheduler.DAGScheduler.handleJobCancellation(DAGScheduler.scala:1955)
	at org.apache.spark.scheduler.DAGSchedulerEventProcessLoop.doOnReceive(DAGScheduler.scala:2205)
	at org.apache.spark.scheduler.DAGSchedulerEventProcessLoop.onReceive(DAGScheduler.scala:2188)
	at org.apache.spark.scheduler.DAGSchedulerEventProcessLoop.onReceive(DAGScheduler.scala:2177)
	at org.apache.spark.util.EventLoop$$anon$1.run(EventLoop.scala:49)
	at org.apache.spark.scheduler.DAGScheduler.runJob(DAGScheduler.scala:775)
	at org.apache.spark.SparkContext.runJob(SparkContext.scala:2114)
	at org.apache.spark.SparkContext.runJob(SparkContext.scala:2135)
	at org.apache.spark.SparkContext.runJob(SparkContext.scala:2154)
	at org.apache.spark.SparkContext.runJob(SparkContext.scala:217